In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
#from boruta import BorutaPy

In [15]:
class TrainDataSet():
    def init(self, headers, filepath, target_key):
        self.headers = headers
        self.filePath = filepath
        self.target_key = target_key
        self.dataSet = None
        self.dataWillBeProcess = None
        self.targetWillBeProcess = None
        self.model = None
        self.y_pred = None
        
    def readCsv(self):
        self.dataSet = pd.read_csv(self.filePath)

    def _validateInputs(self, inputs):
        for key, value in inputs.items():
            if not key.isinstance(value):
                raise Exception("You didn't pass a acceptable argument type to function")

    def outlierAnalysis(self, incomingDataSet, algortihm):
        '''
            Input: 
                [incomingDataSet -> pandas.DataFrame,
                algorithm -> string (like "IQR")]
        '''
        inputs = {incomingDataSet: type(pd.DataFrame), algorithm: str}
        self._validateInputs(inputs)
        if algorithm == 'IQR':
            for key in incomingDataSet.columns[1:]:
                Q1 = incomingDataSet[key].quantile(0.25)
                Q3 = incomingDataSet[key].quantile(0.75)
                IQR = Q3 - Q1
                alt_sinir = Q1 - 1.5 * IQR
                ust_sinir = Q3 + 1.5 * IQR
                incomingDataSet.loc[(
                    incomingDataSet[key] >= alt_sinir
                    ) & (
                        incomingDataSet[key] >= ust_sinir
                    ), key] = None

    def getTargetAndData(self, incomingDataSet):
        try:
            self.dataWillBeProcess = incomingDataSet.drop(self.target_key, axis=1)
            self.targetWillBeProcess = incomingDataSet[self.target_key]
        except KeyError:
            raise Exception("Could not create data or target")
            

        
    def fillNaNFields(self, incomingDataSet, algorithm):
        '''
            Input: 
                [incomingDataSet -> pandas.DataFrame,
                algorithm -> string (like "KNNImputer")]
        '''
        inputs = {incomingDataSet: type(pd.DataFrame), algorithm: str}
        self._validateInputs(inputs)
        df_impute = None
        try:
            if algorithm == 'KNNImputer': 
                from sklearn.impute import KNNImputer
                imputer = KNNImputer(
                    n_neighbors=3, 
                    weights='uniform', 
                    metric='nan_euclidean')
                df_impute = pd.DataFrame(
                    imputer.fit_transform(wine_data), 
                    columns=wine_data.columns
                )
        finally:
            return self.normalizeData(df_impute)

    def normalizeData(self, incomingDataSet):
        '''
            Input: 
                [incomingDataSet -> pandas.DataFrame]
        '''
        inputs = {incomingDataSet, type(pd.DataFrame)}
        self._validateInputs(inputs)
        normalized_dataset = None
        if incomingData=='StandardScaler':
            from sklearn.preprocessing import StandardScaler
            scaler = StandardScaler()
            normalized = scaler.fit_transform(data)
            normalizedDataset = pd.DataFrame(
                normalized, 
                columns=incomingDataSet.columns[1:]
            )
            normalizedDataset.insert(
                0, 
                self.header_key, 
                incomingDataSet[self.header_key], 
                allow_duplicates=True
            )
            return normalized_dataset

    def choiceTestData(self, incomingDataSet, method, splitRate):
        '''
            Input: 
                [incomingDataSet -> pandas.DataFrame, # incomingData
                method -> string (like "train_test_split"),
                splitRate -> float #the proportion of data 
                    to be divided for testing
                ] 
        '''
        inputs = {incomingDataSet: type(pd.DataFrame), method: str, 
                              splitRate: float}
        self._validateInputs(inputs)
        self.getTargetAndData(incomingData)
        if method == 'train_test_split':
            from sklearn.model_selection import train_test_split
            x_train, x_test, y_train, y_test = train_test_split(
                self.dataWillBeProcess, 
                self.targetWillBeProcess, 
                test_size=splitRate, 
                random_state=42
            )
            return x_train, x_test, y_train, y_test


    def trainData(self, trainData, algorithm, x_train, y_train, **kwargs):
        '''
            Input: 
                [incomingDataSet -> pandas.DataFrame,
                algorithm -> string (like "SVC"),
                x_train -> x_training_data,
                y_train -> y_training_data,
                **kwargs -> {
                        dependent variables for algoritm (
                            like SVC_kernel for SVC algorithm
                    )
                ]
        '''
        inputs = {trainData: type(pd.DataFrame)}
        self._validateInputs(inputs)
        if algorithm == 'SVC':
            from sklearn.svm import SVC
            kernel = "linear"
            if kwargs.get("SVC_kernel", None):
                kernel = kwargs.get("SVC_kernel")
            self.model = SVC(kernel=kernel, random_state=42)
            self.model.fit(x_train, y_train)
            self.y_pred = self.model.predict(x_test)

        
    def calculateAccuracy(self, incomingDataSet, method, y_test):
        '''
            Input: 
                [incomingDataSet -> pandas.DataFrame,
                method -> string (like "accuracy_score", "cross_val_score"),
                y_test -> test dataset from choiceTestData method
                ]
        '''
        inputs = {incomingDataSet: type(pd.DataFrame), 'method': str, ytest: list}
        self._validateInputs(inputs)
        if method == 'accuracy_score':
            from sklearn.metrics import accuracy_score
            accuracy = accuracy_score(y_test,self.y_pred)

        elif method == 'cross_val_score':
            from sklearn.model_selection import cross_val_score
            cross_scores = cross_val_score(self.model, incomingDataSet, self.targetWillBeProcess, cv=5)
            print("Cross validation scores for each fold: {}".format(cross_scores)) 
            print("Mean cross validation score : {}".format(cross_scores.mean()))
        
        else:
            raise Exception("You must be choice valid accuracy methods")
        

    def featureSelection(self, normalizedDataSet):
        import seaborn as sns 
        sns.set(rc={"figure.figsize":(15,10)})
        sns.heatmap(normalizedDataSet.iloc[:,:-1].corr(), annot=True, cmap="YlGnBu")
        plt.show()

    def dropFeature(self, normalizedDataSet, key):
        #key = 'OD280/OD315 of diluted wines'
        normalizedDataSet.drop(key, axis=1)
        return normalizedDataSet
        

                    
        

In [ ]:
if __name__ == '__main__':
    file_path = 'wine.data'
    heders = ["Class","Alcohol","Malic acid","Ash","Alcalinity of ash","Magnesium","Total phenols","Flavanoids","Nonflavanoid phenols","Proanthocyanins","Color intensity","Hue","OD280/OD315 of diluted wines","Proline"]
    training = TrainDataSet(headers=headers, filepath=filepath, target_key='Class' )
    training.readCsv()
    training.outlierAnalysis(training.dataSet, 'IQR')
    normalized = training.fillNaNFields(training.dataSet, 'KNNImputer')
    x_train, x_test, y_train, y_test = training.choiceTestData(training.dataSet, 'train_test_split', 0.3)
    training.trainData(x_train, y_train)
    training.calculateAccuracy(training.dataSet, "accuracy_score", y_test)
    training.calculateAccuracy(training.dataSet, "cross_val_score", y_test)
    training.featureSelection(normalized) #TODO: 
    reNormalized = training.dropFeature(normalized, 'OD280/OD315 of diluted wines')
    x_train, x_test, y_train, y_test = training.choiceTestData(reNormalized, 'train_test_split', 0.3)
    training.trainData(x_train, y_train)
    training.calculateAccuracy(training.dataSet, "accuracy_score", y_test)
    training.calculateAccuracy(training.dataSet, "cross_val_score", y_test)
    
    
    
    